In [1]:
import re
import numpy as np
import argparse
import sys

mol = 'lih'
basis = 'sto-3g'

path = './tdcis_data/'
prefix = 'cis-30roots_s0_'

# construct prefix used to load and save files
if basis!='sto-3g' and basis!='6-31g':
    print("Error: basis set not recognized! Must choose either sto-3g or 6-31g")
    sys.exit(1)


ident = prefix+mol+'_'+basis
print(ident)

cis-30roots_s0_lih_sto-3g


In [2]:
def geteigenvalues():
    extract = False
    evals = [0.0]
    f = open(path+ident+'.log','r')
    for x in f:
        if extract:
            strlist = x.split()
            if strlist[0]=='Root':
                evals.append( np.float64(strlist[3]) )
            else:
                extract = False
        if "Excitation Energies [eV] at current iteration:" in x:
            extract = True
    f.close()

    eVenergies = np.array(evals) / 27.211386245988
    return eVenergies 

In [3]:
# n is the length of eVenergies, i.e., the output of geteigenvalues()
# n is a function of the number of AOs (atomic orbitals)
# ne, the number of electrons, which at present must be even
def getCIcoefficients(n, ne=2):
    extract = False
    record = False
    cm = np.zeros((n, n*ne-1))
    cm[0,0] = 1.0
    f = open(path+ident+'.log','r')
    for x in f:
        if "Excitation energies and oscillator strengths:" in x:
            extract = True
            continue
        if extract:
            strlist = x.split()
            if len(strlist)>=3:
                if strlist[0]=='Excited' and strlist[1]=='State':
                    curstate = np.int16(strlist[2][:-1])
                    record = True
                    continue
                if strlist[0]=='SavETr:':
                    extract = False
        if record:
            strlist = x.split()
            if len(strlist)>=3:
                if strlist[1] != '->':
                    record = False
                else:
                    ele = np.int16(strlist[0])
                    ind = np.int16(strlist[2])
                    val = np.float64(strlist[3])
                    # spin-adapted
                    offset = (n-1)*(ele-1)
                    cm[curstate, offset + 2*ind - 3] = val
                    cm[curstate, offset + 2*ind - 2] = -val
    f.close()

    return cm

In [4]:
evals = geteigenvalues()

In [5]:
cimat = getCIcoefficients(evals.shape[0], 4)

In [6]:
cimat.shape

(9, 35)

In [7]:
np.round(cimat @ cimat.T, 2)

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0., -0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0., -0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0., -0., -0.,  0.],
       [ 0., -0.,  0.,  0.,  1., -0.,  0.,  0., -0.],
       [ 0.,  0.,  0.,  0., -0.,  1.,  0.,  0., -0.],
       [ 0.,  0.,  0., -0.,  0.,  0.,  1., -0.,  0.],
       [ 0.,  0., -0., -0.,  0.,  0., -0.,  1.,  0.],
       [ 0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  1.]])